# Import packages

In [1]:
import os
import pandas as pd
from src import features

from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()
ROOT_DIR = os.getenv('ROOT_DIR')

# Load dataset

In [2]:
# Build the full path to the CSV file
csv_file_path = os.path.join(ROOT_DIR, 'app', 'backend', 'database.csv')

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [3]:
df.head()

,Order n°,Isin,Broker,Account,Date,Order_type,Quantity,Unit_price
0,1,LU0131510165,BourseDirect,PEA,01/02/2024,1,1.0,728.74
1,2,LU0131510165,BourseDirect,PEA,04/03/2024,1,1.0,745.87
2,3,LU0131510165,BourseDirect,PEA,05/04/2024,1,1.0,785.92
3,4,LU1964632324,BourseDirect,PEA,30/04/2024,1,1.0,890.24
4,5,LU1832174962,BourseDirect,PEA-PME,01/02/2024,1,5.0,143.89


# Add features

In [5]:
df_features = features.add_column_total(df)
df_features

,Order n°,Isin,Broker,Account,Date,Order_type,Quantity,Unit_price,Total
0,1,LU0131510165,BourseDirect,PEA,01/02/2024,1,1.000000,728.74,728.740000
1,2,LU0131510165,BourseDirect,PEA,04/03/2024,1,1.000000,745.87,745.870000
2,3,LU0131510165,BourseDirect,PEA,05/04/2024,1,1.000000,785.92,785.920000
3,4,LU1964632324,BourseDirect,PEA,30/04/2024,1,1.000000,890.24,890.240000
4,5,LU1832174962,BourseDirect,PEA-PME,01/02/2024,1,5.000000,143.89,719.450000
5,6,LU1832174962,BourseDirect,PEA-PME,05/03/2024,1,5.000000,146.62,733.100000
6,7,LU1832174962,BourseDirect,PEA-PME,05/04/2024,1,4.000000,153.87,615.480000
7,8,LU1832175001,BourseDirect,PEA-PME,09/04/2024,1,1.000000,159.32,159.320000
8,9,US0846707026,TradeRepublic,CTO,16/11/2023,1,0.756200,330.60,249.999720
9,10,US0846707026,TradeRepublic,CTO,04/12/2023,1,0.760340,328.80,249.999792
